# Pytorchによる分類モデル

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [21]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(5, 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, 6)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [22]:
# CSVファイルからデータを読み込む
#データセットの読み込み
TRAIN_DATA_PATH: str = "dataset/train_Mod.csv"
TEST_DATA_PATH: str = "dataset/test.csv"
train_data = pd.read_csv(TRAIN_DATA_PATH)
test_data = pd.read_csv(TEST_DATA_PATH)

#欠損値の補完（とりあえず平均値で）
train_data["knee_depth"] = train_data["knee_depth"].fillna(train_data["knee_depth"].mean())
train_data["hip_depth"] = train_data["hip_depth"].fillna(train_data["hip_depth"].mean())

#idはいらないので削除
train_data = train_data.drop("id", axis=1)
test_data = test_data.drop("id", axis=1)
train_data.isnull().sum()


#学習データを訓練用、検証用に分割
train_data, valid_data = train_test_split(train_data, test_size=0.2, random_state=0)
X_train = train_data.drop("personal_id", axis=1)
y_train = train_data["personal_id"]
X_valid = valid_data.drop("personal_id", axis=1)
y_valid = valid_data["personal_id"]

#ラベルを数値に変換
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_valid = le.transform(y_valid)

print(X_train.values)


[[  9.04029874  32.97510419  10.18393942 -16.02446414  34.92598665]
 [  9.45603037  17.74598672  -2.30618409  22.87743447  29.3612006 ]
 [  9.29508772  25.53407524  -0.6442973   30.34120298  34.5576932 ]
 ...
 [ 11.07121976  34.93547783   6.45382184   8.45650719  39.74778051]
 [ 13.99006782  42.77767379   7.67889063  -4.97488979  33.39031478]
 [  5.62998037  12.8526171    3.96291914  -1.09817854  35.39172352]]


In [23]:
# データをPyTorchのテンソルに変換
X_train = torch.FloatTensor(X_train.values)
y_train = torch.LongTensor(y_train)
X_valid = torch.FloatTensor(X_valid.values)
y_valid = torch.LongTensor(y_valid)

# DataLoaderを作成
train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

In [ ]:
def train_model(model, criterion, optimizer, epoch):
    # 訓練ループ
    for e in range(epoch):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

def test_model(model):
    


In [31]:
best_model = None
best_accuracy = 0

# ハイパーパラメータの候補
learning_rates = [0.001, 0.01, 0.1, 0.2, 0.3, 0.4]
num_epochs = [50, 100, 200]

for lr in learning_rates:
    for epoch in num_epochs:
        model = SimpleNN()
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=lr)

        # 訓練ループ
        for e in range(epoch):
            model.train()
            for batch_idx, (data, target) in enumerate(train_loader):
                optimizer.zero_grad()
                output = model(data)
                loss = criterion(output, target)
                loss.backward()
                optimizer.step()

        # テストループ
        model.eval()
        correct_test = 0
        total_test = 0
        with torch.no_grad():
            for data, target in zip(X_valid, y_valid):
                data = data.unsqueeze(0)
                output = model(data)
                _, predicted = torch.max(output.data, 1)
                total_test += 1
                correct_test += (predicted == target).sum().item()

        test_accuracy = 100 * correct_test / total_test
        print(f'Learning rate: {lr}, Epochs: {epoch}, Test Accuracy: {test_accuracy}%')

        # 最も良いモデルを保存
        if test_accuracy > best_accuracy:
            best_accuracy = test_accuracy
            best_model = model

print(f'Best Model Accuracy: {best_accuracy}%')


Learning rate: 0.001, Epochs: 50, Test Accuracy: 65.04065040650407%
Learning rate: 0.001, Epochs: 100, Test Accuracy: 73.17073170731707%
Learning rate: 0.001, Epochs: 200, Test Accuracy: 71.54471544715447%
Learning rate: 0.01, Epochs: 50, Test Accuracy: 69.10569105691057%
Learning rate: 0.01, Epochs: 100, Test Accuracy: 78.86178861788618%
Learning rate: 0.01, Epochs: 200, Test Accuracy: 70.73170731707317%
Learning rate: 0.1, Epochs: 50, Test Accuracy: 12.195121951219512%
Learning rate: 0.1, Epochs: 100, Test Accuracy: 12.195121951219512%
Learning rate: 0.1, Epochs: 200, Test Accuracy: 39.83739837398374%
Learning rate: 0.2, Epochs: 50, Test Accuracy: 12.195121951219512%
Learning rate: 0.2, Epochs: 100, Test Accuracy: 12.195121951219512%
Learning rate: 0.2, Epochs: 200, Test Accuracy: 12.195121951219512%
Learning rate: 0.3, Epochs: 50, Test Accuracy: 12.195121951219512%
Learning rate: 0.3, Epochs: 100, Test Accuracy: 19.51219512195122%
Learning rate: 0.3, Epochs: 200, Test Accuracy: 12.1